In [1]:
from collections import Counter
import pandas as pd
import numpy as np
import re
from pprint import pprint
import db_mysql as db_mysql

In [2]:
import MySQLdb

hostname = "localhost"
username = "root"
password = ""
db_name  = "drama_ojol"

# Open database connection
db = MySQLdb.connect(hostname,username,password,db_name)
db.set_character_set('utf8')

# prepare a cursor object using cursor() method
cursor = db.cursor()
cursor.execute('SET NAMES utf8;')
cursor.execute('SET CHARACTER SET utf8;')
cursor.execute('SET character_set_connection=utf8;')

def executeSql(sql):
    return cursor.execute(sql);

def fetch(type='all'):
    if type=='all':
        return cursor.fetchall();
    else:
        return cursor.fetchone();
def commit():
    db.commit();

def escapeString(string):
    return MySQLdb.escape_string(string);

# Preprocessing Mentions

In [3]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = re.sub('\.+','.',tweet);
    tweet = re.sub('\,+',',',tweet);
    tweet = re.sub('\"+','"',tweet); #Remove tanda ""
    tweet = tweet.replace("/ ","/");
    tweet = tweet.replace("www. ","www.");
    tweet = re.sub(r'RT', '', tweet);
    tweet = re.sub(r'http\S+', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'@([A-Za-z0-9_]+)', '', tweet, flags=re.MULTILINE);
    tweet = re.sub(r'pic.twitter.com/\S+', '', tweet, flags=re.MULTILINE);
    tweet = tweet.replace("?"," ? ");
    tweet = tweet.replace("-"," ");
    tweet = tweet.replace("/"," ");
    tweet = tweet.replace("["," ");
    tweet = tweet.replace("]"," ");
    tweet = tweet.replace("."," ");
    tweet = tweet.replace(","," ");
    tweet = re.sub(r'[^a-z0-9.,?# -]', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    tweet = re.sub(" \d+", " ", tweet)
    tweet = re.sub(r'( +)', ' ', tweet, flags = re.IGNORECASE|re.MULTILINE)
    #trim
    tweet = tweet.strip('\'"')
    return tweet

# Read the tweets one by one and process it

In [4]:
dokumen = []
docs, index = [], []
fp = open('data_tes.csv', 'r')
line = fp.readline()
while line:
    processedTweet = processTweet(line)
    dokumen.append(processedTweet.split(' '))
    line = fp.readline()
fp.close()

In [5]:
datatest = pd.read_csv("data/kategori/datatest.csv",sep=",", header=None)
datatest.columns = ['kategori']

# print(datatest['kategori'][1])
# if(datasets['kategori'][0]=="aplikas"):
#     print("ada")

# Make dataframe containing list of word each category

In [6]:
df_index = pd.DataFrame({
    'kata':datatest['kategori']
})

# Count word accurance in each document

In [7]:
for i in range(len(dokumen)):
    terdeteksi = []
    for kata in datatest['kategori']:
        if kata in dokumen[i]:
            nilai=1
        else:
            nilai=0
        terdeteksi.append(nilai)
        df_index[i+1] = pd.Series(terdeteksi)

In [8]:
df_index.to_csv("data/matriks_datatest.csv", index=False)
# df_index = pd.read_csv("data/matriks_datatest.csv",sep=",")
# df_index = df_tmp.drop(['kata'], axis=1)
df_index

,kata,1,2,3,4,5,6,7,8,9,...,5293,5294,5295,5296,5297,5298,5299,5300,5301,5302
0,tarif,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tarifnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,murah,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mahal,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,harga,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,argo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,voucher,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,diskon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ongkos,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ongkosnya,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
df_index_baru = df_index.drop(['kata'], axis=1)
df_index_baru

,1,2,3,4,5,6,7,8,9,10,...,5293,5294,5295,5296,5297,5298,5299,5300,5301,5302
0,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Open the aggregration matrix

In [10]:
category = pd.read_csv("data/aggregasi_matriks.csv",sep=",", header=0)
list_cat = category.drop(['kategori'], axis=1)
list_cat

,tarif,tarifnya,murah,mahal,harga,argo,voucher,diskon,ongkos,ongkosnya,...,kompetitor,respon,memecat,recruitment,mengatasnamakan,admin,layanan,penipuan,marketing,sop
0,0.092105,0.017802,0.04257,0.033282,0.06192,0.008514,0.099071,0.0387,0.017802,0.004644,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.002052,0.035916,0.000513,0.001026,0.007696,0.030785,0.042073,0.018984,0.003079,0.00667


In [11]:
list_category = list_cat.values.tolist()
for category in list_category:
    for item in category:
        float(item)

# Assign matrix of each category into variables

In [12]:
cat_harga = []
cat_aplikasi = []
cat_driver = []
cat_penumpang = []
cat_perusahaan = []
cat_harga = np.array(list_category[0])
cat_aplikasi = np.array(list_category[1])
cat_driver = np.array(list_category[2])
cat_penumpang = np.array(list_category[3])
cat_perusahaan = np.array(list_category[4])

# Convert datatest matrix into numpy array

In [13]:
# temp = df_index.drop(['kata'], axis=1)
temp = df_index_baru
temp_datatest = temp.values.tolist()
datatest = np.array(temp_datatest)
# datatest[:,0].shape

In [14]:
temp

,1,2,3,4,5,6,7,8,9,10,...,5293,5294,5295,5296,5297,5298,5299,5300,5301,5302
0,0,1,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fungsi Cosine Similarity

In [15]:
def cos_sim(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Categorisasi dengan cosine

In [18]:
import math
total_label = []
total_datatest = np.size(datatest,1)
for i in range(0,total_datatest):
    harga = cos_sim(datatest[:,i], cat_harga)
    aplikasi = cos_sim(datatest[:,i], cat_aplikasi)
    driver = cos_sim(datatest[:,i], cat_driver)
    penumpang = cos_sim(datatest[:,i], cat_penumpang)
    perusahaan = cos_sim(datatest[:,i], cat_perusahaan)
    maximal = max(harga,aplikasi,driver,penumpang,perusahaan)
    if maximal == harga and math.isnan(maximal)==False:
        label = "harga"
    if maximal == aplikasi and math.isnan(maximal)==False:
        label = "aplikasi"
    elif maximal == driver and math.isnan(maximal)==False:
        label = "driver"
    elif maximal == penumpang and math.isnan(maximal)==False:
        label = "penumpang"
    elif maximal == perusahaan and math.isnan(maximal)==False:
        label = "perusahaan"
    elif math.isnan(maximal)==True:
        label = "others"
#     print(3399+i+1,"=",label,maximal)
    print(i+1,"=",label,maximal)   
   
    
    sql = '''UPDATE opinions
    		 SET cat_result="%s" WHERE id=%d''' % \
    		 (label,i+1);
    executeSql(sql);
    commit();

1 = penumpang 0.873138537488504
2 = harga 0.24773886028426761
3 = driver 0.9902528390930346
4 = perusahaan 0.4501968055113169
5 = driver 0.40426902868671827
6 = driver 0.9902528390930346
7 = harga 0.2573600183788746
8 = driver 0.5717227432161548
9 = perusahaan 0.4046320034467727
10 = perusahaan 0.4501968055113169
11 = penumpang 0.6174021807734257
12 = penumpang 0.5658609368595506
13 = aplikasi 0.1412225575365629
14 = aplikasi 0.34184990679839916
15 = driver 0.9902528390930346
16 = aplikasi 0.244604644828146
17 = others nan
18 = driver 0.4951264195465173
19 = aplikasi 0.4390951687686523
20 = perusahaan 0.6366744280911469
21 = driver 0.9902528390930346
22 = perusahaan 0.31833721404557347
23 = aplikasi 0.3132304260540276
24 = penumpang 0.6174021807734257
25 = perusahaan 0.6366744280911469
26 = others nan
27 = driver 0.5717227432161548
28 = penumpang 0.1141796548570843
29 = aplikasi 0.5512553474590951
30 = aplikasi 0.08680020559729004
31 = aplikasi 0.4390951687686523


C:\Users\Brandon\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


32 = driver 0.7002144976119159
33 = aplikasi 0.6066311856396361
34 = driver 0.5717227432161548
35 = driver 0.9902528390930346
36 = driver 0.7002144976119159
37 = driver 0.5717227432161548
38 = penumpang 0.6174021807734257
39 = penumpang 0.0756160156186688
40 = others nan
41 = driver 0.5717227432161548
42 = driver 0.4845710053570147
43 = perusahaan 0.6366744280911469
44 = driver 0.0031620632582608477
45 = driver 0.006324126516521695
46 = penumpang 0.6174021807734257
47 = driver 0.7002144976119159
48 = driver 0.5717227432161548
49 = aplikasi 0.244604644828146
50 = aplikasi 0.6066311856396361
51 = driver 0.02582351705779191
52 = driver 0.7661740325029098
53 = driver 0.9902528390930346
54 = perusahaan 0.6366744280911469
55 = driver 0.9902528390930346
56 = others nan
57 = penumpang 0.6838916457977969
58 = driver 0.07220044664026429
59 = driver 0.9902528390930346
60 = others nan
61 = others nan
62 = others nan
63 = penumpang 0.12021007532782896
64 = aplikasi 0.30626859592109407
65 = driver 0

303 = perusahaan 0.6210240974195099
304 = aplikasi 0.24214624429479262
305 = aplikasi 0.09784185793125841
306 = driver 0.7002144976119159
307 = driver 0.9902528390930346
308 = aplikasi 0.4390951687686523
309 = others nan
310 = aplikasi 0.09784185793125841
311 = driver 0.11701296031502226
312 = driver 0.4951264195465173
313 = perusahaan 0.4501968055113169
314 = harga 0.5793639541469566
315 = perusahaan 0.4501968055113169
316 = driver 0.5417668539518499
317 = aplikasi 0.34254834723571626
318 = aplikasi 0.3239866145287499
319 = others nan
320 = driver 0.9902528390930346
321 = aplikasi 0.17062665718820277
322 = driver 0.7002144976119159
323 = aplikasi 0.23215610732780687
324 = aplikasi 0.0047727747660653505
325 = driver 0.5168600266957102
326 = aplikasi 0.33135634550695875
327 = others nan
328 = harga 0.17301513969740057
329 = perusahaan 0.36758415244457476
330 = others nan
331 = harga 0.704985273704454
332 = others nan
333 = aplikasi 0.09784185793125841
334 = penumpang 0.2505228083170405


590 = perusahaan 0.37598883470216465
591 = perusahaan 0.6366744280911469
592 = perusahaan 0.04010547515440522
593 = driver 0.5417668539518499
594 = driver 0.7002144976119159
595 = driver 0.7002144976119159
596 = harga 0.16829002015430194
597 = perusahaan 0.4501968055113169
598 = aplikasi 0.4390951687686523
599 = driver 0.7002144976119159
600 = harga 0.13360332647788578
601 = driver 0.7002144976119159
602 = driver 0.4428545326248373
603 = driver 0.7002144976119159
604 = penumpang 0.6174021807734257
605 = others nan
606 = driver 0.5546786124807458
607 = others nan
608 = driver 0.7002144976119159
609 = harga 0.3021682698632111
610 = driver 0.7002144976119159
611 = aplikasi 0.1073874074633891
612 = driver 0.5866319577408939
613 = perusahaan 0.6943260487477381
614 = harga 0.2573600183788746
615 = penumpang 0.6174021807734257
616 = aplikasi 0.37346953166080593
617 = aplikasi 0.18308618477173244
618 = perusahaan 0.36758415244457476
619 = penumpang 0.49365909617279413
620 = harga 0.24773886028

...

4095 = driver 0.9902528390930346
4096 = perusahaan 0.6366744280911469
4097 = others nan
4098 = harga 0.06055529959263882
4099 = others nan
4100 = penumpang 0.6174021807734257
4101 = others nan
4102 = driver 0.7002144976119159
4103 = driver 0.5717227432161548
4104 = driver 0.5717227432161548
4105 = aplikasi 0.2945539945788818
4106 = others nan
4107 = others nan
4108 = aplikasi 0.1618892723765846
4109 = perusahaan 0.04750107969207394
4110 = driver 0.9902528390930346
4111 = driver 0.6134076905195226
4112 = aplikasi 0.04809176371972608
4113 = others nan
4114 = others nan
4115 = others nan
4116 = driver 0.040579812711670854
4117 = perusahaan 0.08221622553213519
4118 = aplikasi 0.34254834723571626
4119 = perusahaan 0.35342950504994547
4120 = driver 0.4951264195465173
4121 = harga 0.27682422072164636
4122 = perusahaan 0.4501968055113169
4123 = driver 0.09328086881066494
4124 = aplikasi 0.4390951687686523
4125 = others nan
4126 = aplikasi 0.4844365184292383
4127 = perusahaan 0.0370975642735574

4379 = perusahaan 0.434154509855177
4380 = aplikasi 0.20417906227908994
4381 = perusahaan 0.531728509336657
4382 = perusahaan 0.531728509336657
4383 = driver 0.7002144976119159
4384 = driver 0.7002144976119159
4385 = others nan
4386 = driver 0.9902528390930346
4387 = aplikasi 0.4656899960535984
4388 = penumpang 0.873138537488504
4389 = perusahaan 0.531728509336657
4390 = aplikasi 0.25892296558733047
4391 = others nan
4392 = driver 0.0500660024864534
4393 = perusahaan 0.7519776694043293
4394 = perusahaan 0.531728509336657
4395 = others nan
4396 = others nan
4397 = driver 0.4951264195465173
4398 = others nan
4399 = perusahaan 0.7519776694043293
4400 = driver 0.7002144976119159
4401 = perusahaan 0.7519776694043293
4402 = perusahaan 0.531728509336657
4403 = driver 0.9902528390930346
4404 = driver 0.9902528390930346
4405 = harga 0.5092409900354963
4406 = perusahaan 0.7519776694043293
4407 = aplikasi 0.4331292020805764
4408 = harga 0.4652957743555704
4409 = driver 0.9902528390930346
4410 = p

4647 = others nan
4648 = driver 0.7002144976119159
4649 = perusahaan 0.3080458875785089
4650 = aplikasi 0.34254834723571626
4651 = perusahaan 0.31833721404557347
4652 = aplikasi 0.37346953166080593
4653 = others nan
4654 = driver 0.9902528390930346
4655 = others nan
4656 = perusahaan 0.6943260487477381
4657 = driver 0.9902528390930346
4658 = others nan
4659 = penumpang 0.873138537488504
4660 = others nan
4661 = aplikasi 0.49531228895889556
4662 = aplikasi 0.172961603067709
4663 = driver 0.7002144976119159
4664 = driver 0.4428545326248373
4665 = others nan
4666 = driver 0.5717227432161548
4667 = perusahaan 0.36758415244457476
4668 = others nan
4669 = others nan
4670 = aplikasi 0.008352354071108558
4671 = driver 0.7002144976119159
4672 = perusahaan 0.49273508636879226
4673 = others nan
4674 = aplikasi 0.6066311856396361
4675 = harga 0.3960763251440984
4676 = perusahaan 0.4501968055113169
4677 = others nan
4678 = driver 0.9902528390930346
4679 = driver 0.7002144976119159
4680 = driver 0.9

4911 = others nan
4912 = aplikasi 0.172961603067709
4913 = penumpang 0.5264017914021583
4914 = penumpang 0.02350757530779848
4915 = driver 0.6255784779342344
4916 = others nan
4917 = driver 0.7002144976119159
4918 = driver 0.6255784779342344
4919 = aplikasi 0.09784185793125841
4920 = harga 0.7201755146245009
4921 = harga 0.2573600183788746
4922 = driver 0.5717227432161548
4923 = harga 0.5092409900354963
4924 = driver 0.7002144976119159
4925 = perusahaan 0.9819253148479739
4926 = harga 0.21726148179684068
4927 = perusahaan 0.4501968055113169
4928 = driver 0.9902528390930346
4929 = perusahaan 0.7519776694043293
4930 = others nan
4931 = perusahaan 0.6366744280911469
4932 = others nan
4933 = others nan
4934 = perusahaan 0.6366744280911469
4935 = others nan
4936 = others nan
4937 = driver 0.5717227432161548
4938 = driver 0.9902528390930346
4939 = aplikasi 0.244604644828146
4940 = driver 0.7002144976119159
4941 = others nan
4942 = others nan
4943 = penumpang 0.436569268744252
4944 = driver 0

5208 = driver 0.7002144976119159
5209 = aplikasi 0.4893547806928265
5210 = harga 0.5793639541469566
5211 = others nan
5212 = others nan
5213 = harga 0.7201755146245009
5214 = harga 0.27682422072164636
5215 = others nan
5216 = penumpang 0.873138537488504
5217 = perusahaan 0.4170969474682322
5218 = others nan
5219 = driver 0.5417668539518499
5220 = harga 0.5092409900354963
5221 = driver 0.5717227432161548
5222 = driver 0.11701296031502226
5223 = others nan
5224 = harga 0.28331229003203334
5225 = driver 0.6255784779342344
5226 = driver 0.7002144976119159
5227 = others nan
5228 = driver 0.9902528390930346
5229 = aplikasi 0.18308618477173244
5230 = driver 0.7002144976119159
5231 = others nan
5232 = perusahaan 0.7519776694043293
5233 = driver 0.7289087583596184
5234 = perusahaan 0.531728509336657
5235 = driver 0.7002144976119159
5236 = driver 0.7002144976119159
5237 = perusahaan 0.7519776694043293
5238 = perusahaan 0.6479276684710225
5239 = driver 0.9902528390930346
5240 = harga 0.0929956377